# Version

* `v7`: yolov5x_fold4_finetune768 tta768 conf_thr=0.01
* `v6`: yolov5x_fold3_finetune768 tta768 conf_thr=0.01
* `v5`: yolov5x_fold2_finetune768 tta768 conf_thr=0.01
* `v4`: yolov5x_fold1_finetune768 tta768 conf_thr=0.01
* `v3`: yolov5x_fold0_finetune768 tta768 conf_thr=0.01

# [Training Notebook](https://www.kaggle.com/awsaf49/vinbigdata-cxr-ad-yolov5-14-class)
* Select `GPU` as the **Accelerator**

In [ ]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns

In [ ]:
dim = 1024 #1024, 256, 'original'
test_dir = f'/kaggle/input/vinbigdata-{dim}-image-dataset/vinbigdata/test'
# yolov5x Fold 4 finetune768
weights_dir = '/kaggle/input/vinbigdata-final-models/yolov5x_fold4_finetune768_best.pt'

In [ ]:
test_df = pd.read_csv(f'/kaggle/input/vinbigdata-{dim}-image-dataset/vinbigdata/test.csv')
test_df.head()

# YOLOv5 Stuff

In [ ]:
shutil.copytree('/kaggle/input/yolov5-official-v40/multilabel-YOLOv5-v4.0/YOLOv5-v4.0', '/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5') # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
!pip install -U -r /kaggle/working/yolov5/requirements.txt # install dependencies

In [ ]:
!pip install pycocotools
!pip install thop

In [ ]:
!pip install torchvision==0.8.1

In [ ]:
!pip install --upgrade seaborn
!pip install --upgrade matplotlib

# Inference

In [ ]:
!python detect.py --weights $weights_dir\
--img 768\
--conf 0.01\
--iou 0.5\
--source $test_dir\
--save-txt --save-conf --exist-ok --augment

# Process Submission

In [ ]:
!pip install pandas==1.1.5

In [ ]:
def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes

In [ ]:
image_ids = []
PredictionStrings = []

for file_path in tqdm(glob('runs/detect/exp/labels/*txt')):
    image_id = file_path.split('/')[-1].split('.')[0]
    w, h = test_df.loc[test_df.image_id==image_id,['width', 'height']].values[0]
    f = open(file_path, 'r')
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]
#     bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 1).astype(str))
    bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 3).astype(str))
    for idx in range(len(bboxes)):
        bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
    image_ids.append(image_id)
    PredictionStrings.append(' '.join(bboxes))

In [ ]:
pred_df = pd.DataFrame({'image_id':image_ids,
                        'PredictionString':PredictionStrings})
sub_df = pd.merge(test_df, pred_df, on = 'image_id', how = 'left').fillna("14 1 0 0 1 1")
sub_df = sub_df[['image_id', 'PredictionString']]
sub_df.to_csv('/kaggle/working/yolov5x_fold4_finetune768_submission.csv',index = False)
sub_df.tail()

In [ ]:
len(sub_df)

In [ ]:
shutil.rmtree('/kaggle/working/yolov5')